# Chapter 11: Working with Schemas (Namespaces)

PostgreSQL schemas provide logical namespaces within a database, enabling multi-tenancy, service boundaries, and security isolation without the overhead of separate database connections. This chapter covers architectural patterns, security configurations, and operational practices for production schema management.

## 11.1 Schema Fundamentals and Architecture

### 11.1.1 Understanding Schemas as Namespaces

A schema is a named collection of database objects (tables, views, functions, indexes). Schemas prevent naming collisions and enable logical separation within a single database.

```sql
-- Schema creation and basic management
CREATE SCHEMA IF NOT EXISTS ecommerce;
CREATE SCHEMA IF NOT EXISTS analytics;
CREATE SCHEMA IF NOT EXISTS audit;

-- Schema-qualified object references
CREATE TABLE ecommerce.orders (
    order_id BIGSERIAL PRIMARY KEY,
    total_cents INTEGER NOT NULL
);

CREATE TABLE analytics.orders_summary (
    order_date DATE PRIMARY KEY,
    daily_revenue BIGINT NOT NULL
);

-- Unqualified reference depends on search_path
SELECT * FROM orders;        -- Ambiguous: which schema?
SELECT * FROM ecommerce.orders;  -- Explicit: unambiguous, recommended

-- Schema boundaries are logical, not physical
-- Tables in different schemas can have foreign key relationships
ALTER TABLE ecommerce.order_items
ADD CONSTRAINT fk_order 
    FOREIGN KEY (order_id) 
    REFERENCES ecommerce.orders(order_id);

-- Cross-schema foreign keys (use with caution)
CREATE SCHEMA inventory;
CREATE TABLE inventory.products (
    sku VARCHAR(50) PRIMARY KEY
);

-- Referencing product from ecommerce schema
ALTER TABLE ecommerce.order_items
ADD COLUMN product_sku VARCHAR(50) REFERENCES inventory.products(sku);
-- Valid: FKs work across schemas in same database
```

### 11.1.2 The Public Schema and Security Hardening

By default, PostgreSQL creates a `public` schema with permissive access. Production systems should restrict or eliminate this default.

```sql
-- Security audit: Check public schema privileges
SELECT nspname, nspacl 
FROM pg_namespace 
WHERE nspname = 'public';

-- Revoke unsafe defaults (run as superuser or schema owner)
-- Step 1: Revoke create privileges from public (all users)
REVOKE CREATE ON SCHEMA public FROM PUBLIC;

-- Step 2: If using schema-per-tenant, consider dropping public entirely
-- DROP SCHEMA public CASCADE;  -- Destructive: removes all objects in public

-- Alternative: Lock down public for specific roles only
REVOKE ALL ON SCHEMA public FROM PUBLIC;
GRANT USAGE ON SCHEMA public TO app_role;
-- Now only app_role can see public schema objects

-- Best practice: Never store application tables in public
-- Use explicit schemas for all business logic
```

## 11.2 Multi-Tenant Architecture Patterns

### 11.2.1 Schema-Per-Tenant Isolation

Schema-per-tenant provides strong isolation with shared infrastructure, suitable for B2B SaaS with strict data separation requirements.

```sql
-- Tenant schema provisioning template
CREATE OR REPLACE FUNCTION provision_tenant(tenant_id TEXT)
RETURNS void AS $$
DECLARE
    schema_name TEXT := 'tenant_' || tenant_id;
BEGIN
    -- Create isolated schema
    EXECUTE format('CREATE SCHEMA IF NOT EXISTS %I', schema_name);
    
    -- Create tenant tables within schema
    EXECUTE format('
        CREATE TABLE %I.users (
            user_id BIGSERIAL PRIMARY KEY,
            email VARCHAR(255) UNIQUE NOT NULL,
            created_at TIMESTAMPTZ DEFAULT NOW()
        )', schema_name);
    
    EXECUTE format('
        CREATE TABLE %I.documents (
            document_id UUID PRIMARY KEY DEFAULT gen_random_uuid(),
            user_id BIGINT NOT NULL REFERENCES %I.users(user_id),
            content TEXT,
            created_at TIMESTAMPTZ DEFAULT NOW()
        )', schema_name, schema_name);
    
    -- Set ownership for tenant-specific admin role
    EXECUTE format('ALTER SCHEMA %I OWNER TO tenant_admin_role', schema_name);
    
    -- Grant limited privileges to application role
    EXECUTE format('GRANT USAGE ON SCHEMA %I TO app_service_role', schema_name);
    EXECUTE format('GRANT SELECT, INSERT, UPDATE, DELETE ON ALL TABLES IN SCHEMA %I TO app_service_role', schema_name);
    
    -- Set default privileges for future objects
    EXECUTE format('ALTER DEFAULT PRIVILEGES IN SCHEMA %I 
                    GRANT SELECT, INSERT, UPDATE, DELETE ON TABLES TO app_service_role', schema_name);
END;
$$ LANGUAGE plpgsql;

-- Usage
SELECT provision_tenant('acme_corp');
SELECT provision_tenant('globex_inc');

-- Result: tenant_acme_corp.users and tenant_globex_inc.users are completely separate tables
```

### 11.2.2 Schema Isolation vs Row-Level Security (RLS)

Choosing between schema-per-tenant and RLS depends on scale and operational complexity.

```sql
-- Row-Level Security approach (alternative to schema-per-tenant)
CREATE TABLE documents_shared (
    document_id UUID PRIMARY KEY DEFAULT gen_random_uuid(),
    tenant_id TEXT NOT NULL,  -- Discriminator column
    user_id BIGINT NOT NULL,
    content TEXT,
    created_at TIMESTAMPTZ DEFAULT NOW()
);

-- Enable RLS
ALTER TABLE documents_shared ENABLE ROW LEVEL SECURITY;

-- Policy: Users see only their tenant's data
CREATE POLICY tenant_isolation ON documents_shared
    USING (tenant_id = current_setting('app.current_tenant')::TEXT);

-- Application sets context per request
SET LOCAL app.current_tenant = 'acme_corp';
-- Now SELECT * FROM documents_shared returns only acme_corp rows

-- Comparison:
-- Schema-per-tenant:
--   Pros: True physical separation, easy per-tenant backup/restore, custom schema migrations per tenant
--   Cons: Connection pooling complexity, schema proliferation (1000s of schemas impacts performance), harder cross-tenant analytics

-- RLS approach:
--   Pros: Single query plan cache, simpler connection pooling, easier aggregation across tenants
--   Cons: Risk of data leakage if policy bypassed, harder to customize schema per tenant, vacuum overhead on large single tables

-- Hybrid approach: Schema-per-tenant for isolation, RLS within shared analytics schema
```

### 11.2.3 Connection Routing and Schema Context

Applications must set the correct schema context without relying on search_path vulnerabilities.

```sql
-- Anti-pattern: Relying on search_path for tenant isolation
SET search_path = tenant_acme_corp, public;
SELECT * FROM users;  -- Dangerous: assumes correct search_path

-- Industry standard: Explicit schema qualification in application
-- Application code (pseudocode):
-- tenant_schema = f"tenant_{current_tenant_id}"
-- db.query(f"SELECT * FROM {tenant_schema}.users WHERE user_id = %s", [user_id])

-- Using session variables for schema context (safer than search_path)
SET app.current_schema = 'tenant_acme_corp';

-- Views that respect session context
CREATE OR REPLACE VIEW current_tenant_users AS
SELECT * 
FROM users 
WHERE table_schema = current_setting('app.current_schema', true);

-- Better: Use SECURITY DEFINER functions with schema parameter
CREATE OR REPLACE FUNCTION get_tenant_users(tenant_schema TEXT)
RETURNS TABLE (user_id BIGINT, email TEXT) AS $$
BEGIN
    RETURN QUERY EXECUTE format(
        'SELECT user_id, email FROM %I.users WHERE deleted_at IS NULL',
        tenant_schema
    );
END;
$$ LANGUAGE plpgsql SECURITY DEFINER;
-- Application calls: SELECT * FROM get_tenant_users('tenant_acme_corp');
```

## 11.3 Bounded Contexts and Microservices

### 11.3.1 Schema as Service Boundary

In microservice architectures, schemas can represent service boundaries within a shared database (modular monolith transition pattern).

```sql
-- Service: User Management (bounded context)
CREATE SCHEMA user_service;

CREATE TABLE user_service.profiles (
    user_id BIGSERIAL PRIMARY KEY,
    email VARCHAR(255) UNIQUE NOT NULL,
    encrypted_password VARCHAR(255) NOT NULL,
    email_verified_at TIMESTAMPTZ,
    created_at TIMESTAMPTZ DEFAULT NOW()
);

CREATE TABLE user_service.sessions (
    session_token UUID PRIMARY KEY DEFAULT gen_random_uuid(),
    user_id BIGINT REFERENCES user_service.profiles(user_id) ON DELETE CASCADE,
    expires_at TIMESTAMPTZ NOT NULL,
    created_at TIMESTAMPTZ DEFAULT NOW()
);

-- Service: Billing (separate bounded context)
CREATE SCHEMA billing_service;

CREATE TABLE billing_service.accounts (
    account_id BIGSERIAL PRIMARY KEY,
    user_id BIGINT NOT NULL,  -- Logical reference, no FK constraint across contexts
    balance_cents INTEGER DEFAULT 0,
    currency CHAR(3) DEFAULT 'USD',
    last_invoice_at TIMESTAMPTZ
);

-- Anti-corruption layer: View into other context (read-only)
CREATE VIEW billing_service.user_emails AS
SELECT user_id, email 
FROM user_service.profiles;

-- Event publishing table (outbox pattern)
CREATE SCHEMA outbox;

CREATE TABLE outbox.events (
    id BIGSERIAL PRIMARY KEY,
    aggregate_type VARCHAR(100) NOT NULL,
    aggregate_id VARCHAR(100) NOT NULL,
    event_type VARCHAR(100) NOT NULL,
    payload JSONB NOT NULL,
    created_at TIMESTAMPTZ DEFAULT NOW(),
    published_at TIMESTAMPTZ,
    UNIQUE(aggregate_type, aggregate_id, event_type, created_at)
);

-- Service-specific grants
GRANT USAGE ON SCHEMA user_service TO user_service_role;
GRANT SELECT, INSERT, UPDATE, DELETE ON ALL TABLES IN SCHEMA user_service TO user_service_role;

GRANT USAGE ON SCHEMA billing_service TO billing_service_role;
GRANT SELECT, INSERT, UPDATE ON ALL TABLES IN SCHEMA billing_service TO billing_service_role;
GRANT SELECT ON billing_service.user_emails TO billing_service_role;  -- Read-only view access
```

### 11.3.2 API Schemas (External Interfaces)

Expose only specific views/functions to external consumers while keeping internal tables private.

```sql
-- Internal schema (private)
CREATE SCHEMA internal;

CREATE TABLE internal.customers (
    customer_id BIGSERIAL PRIMARY KEY,
    ssn VARCHAR(11) ENCRYPTED,  -- Sensitive
    internal_notes TEXT,        -- Sensitive
    full_name VARCHAR(255),
    email VARCHAR(255)
);

-- Public API schema (controlled exposure)
CREATE SCHEMA api;

-- Expose only safe columns
CREATE VIEW api.customers AS
SELECT customer_id, full_name, email, created_at
FROM internal.customers
WHERE deleted_at IS NULL;

-- Expose functionality via functions, not direct table access
CREATE FUNCTION api.create_customer(p_name TEXT, p_email TEXT)
RETURNS BIGINT AS $$
DECLARE
    new_id BIGINT;
BEGIN
    INSERT INTO internal.customers (full_name, email)
    VALUES (p_name, p_email)
    RETURNING customer_id INTO new_id;
    
    RETURN new_id;
END;
$$ LANGUAGE plpgsql SECURITY DEFINER;

-- Grant external service access only to api schema
GRANT USAGE ON SCHEMA api TO partner_service_role;
GRANT SELECT ON api.customers TO partner_service_role;
GRANT EXECUTE ON FUNCTION api.create_customer TO partner_service_role;

-- Internal service has full access
GRANT ALL ON SCHEMA internal TO internal_service_role;
```

## 11.4 Search Path and Security Configuration

### 11.4.1 Understanding Search Path Mechanics

The `search_path` determines which schemas are checked for unqualified object names. Misconfiguration leads to security vulnerabilities and unexpected behavior.

```sql
-- Default search_path: "$user", public
SHOW search_path;  -- Usually: "user", public

-- Setting search_path (session level)
SET search_path = ecommerce, analytics, public;

-- Query resolution order:
-- 1. Look in schema named same as current user (if exists)
-- 2. Look in ecommerce schema
-- 3. Look in analytics schema
-- 4. Look in public schema

-- Danger: Schema squatting attack
-- If user 'ecommerce' exists and creates table 'users', 
-- and search_path includes "$user", 
-- unqualified SELECT * FROM users might hit the wrong table!

-- Security-hardened search_path (production standard)
ALTER DATABASE production_db SET search_path = "$user";
-- Forces explicit qualification or use of user-named schema only

-- Or set at role level
ALTER ROLE app_role SET search_path = '';
-- Empty search_path forces fully qualified names

-- Function-specific search_path (critical for security)
CREATE OR REPLACE FUNCTION public.calculate_total()
RETURNS INTEGER AS $$
BEGIN
    -- Without SET search_path, this looks in caller's search_path (dangerous)
    RETURN (SELECT SUM(amount) FROM invoices);  
END;
$$ LANGUAGE plpgsql
SET search_path = accounting, pg_temp;  -- Explicit, limited search path
-- pg_temp included for temporary tables, but explicitly listed
```

### 11.4.2 Schema Squatting and Privilege Escalation Prevention

```sql
-- Attack scenario prevention:
-- 1. Attacker creates schema named same as target user
-- 2. Creates trojan table/function in that schema
-- 3. Waits for target user to run unqualified query

-- Defense: Revoke schema creation from untrusted users
REVOKE CREATE ON SCHEMA public FROM PUBLIC;
REVOKE ALL ON SCHEMA public FROM PUBLIC;

-- Audit existing schemas for suspicious names matching usernames
SELECT n.nspname, r.rolname 
FROM pg_namespace n
JOIN pg_roles r ON n.nspname = r.rolname
WHERE n.nspname NOT IN ('pg_catalog', 'information_schema', 'public');

-- Check for unauthorized object creation
SELECT schemaname, tablename, tableowner 
FROM pg_tables 
WHERE schemaname = 'public' 
  AND tableowner NOT IN ('postgres', 'admin_role');
```

## 11.5 Ownership and Privilege Management

### 11.5.1 Schema-Level Privileges

```sql
-- Create schema with specific owner
CREATE SCHEMA marketing AUTHORIZATION marketing_admin;

-- Privilege types:
-- USAGE: Access objects within schema (required to use any object)
-- CREATE: Create new objects within schema

-- Typical application role setup
CREATE ROLE readonly_role;
CREATE ROLE readwrite_role;
CREATE ROLE admin_role;

-- Schema usage (required for all access)
GRANT USAGE ON SCHEMA marketing TO readonly_role;
GRANT USAGE, CREATE ON SCHEMA marketing TO readwrite_role;
GRANT ALL ON SCHEMA marketing TO admin_role;

-- Object privileges within schema
GRANT SELECT ON ALL TABLES IN SCHEMA marketing TO readonly_role;
GRANT SELECT, INSERT, UPDATE, DELETE ON ALL TABLES IN SCHEMA marketing TO readwrite_role;
GRANT ALL ON ALL TABLES IN SCHEMA marketing TO admin_role;

-- Sequences (for serial/identity columns)
GRANT USAGE, SELECT ON ALL SEQUENCES IN SCHEMA marketing TO readwrite_role;

-- Future objects: Default privileges
ALTER DEFAULT PRIVILEGES IN SCHEMA marketing
    GRANT SELECT ON TABLES TO readonly_role;
    
ALTER DEFAULT PRIVILEGES IN SCHEMA marketing
    GRANT SELECT, INSERT, UPDATE, DELETE ON TABLES TO readwrite_role;
    
ALTER DEFAULT PRIVILEGES IN SCHEMA marketing
    GRANT USAGE, SELECT ON SEQUENCES TO readwrite_role;

-- Revocation cascade
REVOKE ALL ON SCHEMA marketing FROM PUBLIC;  -- Remove default access
```

### 11.5.2 Schema-Bound Roles and Security Contexts

```sql
-- Create schema-specific roles (defense in depth)
CREATE ROLE tenant_acme_admin;
CREATE ROLE tenant_acme_app;

-- Bind role to schema via default privileges
SET ROLE tenant_acme_admin;
ALTER DEFAULT PRIVILEGES IN SCHEMA tenant_acme
    GRANT SELECT, INSERT, UPDATE, DELETE ON TABLES TO tenant_acme_app;
SET ROLE postgres;  -- Restore superuser

-- Row-Level Security with schema context
CREATE POLICY tenant_schema_isolation ON tenant_acme.orders
    FOR ALL
    TO tenant_acme_app
    USING (true);  -- Role already scoped to schema via grants

-- Cross-schema access control
-- Allow analytics role to read specific tables from multiple schemas
GRANT SELECT ON ecommerce.orders TO analytics_role;
GRANT SELECT ON crm.customers TO analytics_role;

-- But prevent analytics from seeing sensitive schemas
-- (Absence of GRANT USAGE means no access, even if they guess object names)
```

## 11.6 Cross-Schema Patterns and Integration

### 11.6.1 Cross-Schema Views and Reporting

```sql
-- Analytics schema aggregating data from operational schemas
CREATE SCHEMA data_warehouse;

CREATE VIEW data_warehouse.daily_revenue AS
SELECT 
    DATE_TRUNC('day', o.created_at) as day,
    SUM(o.total_cents) / 100.0 as revenue_usd,
    COUNT(*) as order_count
FROM ecommerce.orders o
JOIN ecommerce.order_items oi ON o.order_id = oi.order_id
GROUP BY 1;

-- Materialized view across schemas (for performance)
CREATE MATERIALIZED VIEW data_warehouse.customer_360 AS
SELECT 
    c.customer_id,
    c.email,
    c.full_name,
    COUNT(DISTINCT o.order_id) as total_orders,
    SUM(o.total_cents) as lifetime_value_cents
FROM crm.customers c
LEFT JOIN ecommerce.orders o ON c.customer_id = o.customer_id
GROUP BY c.customer_id, c.email, c.full_name;

-- Refresh strategy
REFRESH MATERIALIZED VIEW CONCURRENTLY data_warehouse.customer_360;
```

### 11.6.2 Cross-Schema Foreign Keys and Integrity

While possible, cross-schema foreign keys create tight coupling. Use with architectural awareness.

```sql
-- Schema A: Orders (operational)
CREATE SCHEMA sales;
CREATE TABLE sales.orders (
    order_id BIGSERIAL PRIMARY KEY,
    customer_id BIGINT NOT NULL,  -- References CRM, but no FK
    order_date TIMESTAMPTZ DEFAULT NOW()
);

-- Schema B: Customers (master data)
CREATE SCHEMA crm;
CREATE TABLE crm.customers (
    customer_id BIGSERIAL PRIMARY KEY,
    email VARCHAR(255) UNIQUE
);

-- Option 1: Foreign key across schemas (tight coupling)
ALTER TABLE sales.orders
ADD CONSTRAINT fk_customer 
    FOREIGN KEY (customer_id) 
    REFERENCES crm.customers(customer_id)
    ON DELETE RESTRICT;  -- Prevent deletion of customers with orders

-- Option 2: Soft reference with application validation (loose coupling)
-- No FK constraint, but application validates existence
-- Better for microservices separation, eventual consistency

-- Option 3: Trigger-based validation (middle ground)
CREATE OR REPLACE FUNCTION validate_customer_exists()
RETURNS TRIGGER AS $$
BEGIN
    IF NOT EXISTS (SELECT 1 FROM crm.customers WHERE customer_id = NEW.customer_id) THEN
        RAISE EXCEPTION 'Customer % does not exist in CRM schema', NEW.customer_id;
    END IF;
    RETURN NEW;
END;
$$ LANGUAGE plpgsql;

CREATE TRIGGER trg_validate_customer
    BEFORE INSERT OR UPDATE ON sales.orders
    FOR EACH ROW
    EXECUTE FUNCTION validate_customer_exists();
```

## 11.7 Operational Considerations

### 11.7.1 Schema Maintenance and Migrations

```sql
-- Schema versioning pattern
CREATE SCHEMA schema_migrations;

CREATE TABLE schema_migrations.applied_migrations (
    schema_name VARCHAR(63) NOT NULL,
    migration_version VARCHAR(20) NOT NULL,
    applied_at TIMESTAMPTZ DEFAULT NOW(),
    checksum VARCHAR(64),
    PRIMARY KEY (schema_name, migration_version)
);

-- Per-schema migration tracking
INSERT INTO schema_migrations.applied_migrations 
VALUES ('tenant_acme', '001_initial', NOW(), 'abc123...');

-- Bulk schema changes (use with caution)
DO $$
DECLARE
    tenant_schema TEXT;
BEGIN
    FOR tenant_schema IN 
        SELECT schema_name 
        FROM information_schema.schemata 
        WHERE schema_name LIKE 'tenant_%'
    LOOP
        EXECUTE format('ALTER TABLE %I.users ADD COLUMN last_login TIMESTAMPTZ', tenant_schema);
        
        INSERT INTO schema_migrations.applied_migrations (schema_name, migration_version)
        VALUES (tenant_schema, '002_add_last_login');
    END LOOP;
END $$;
```

### 11.7.2 Backup and Restore Strategies

```sql
-- Dump specific schema only
pg_dump -h localhost -U postgres -n ecommerce -f ecommerce_schema.sql production_db

-- Dump multiple schemas
pg_dump -h localhost -U postgres -n 'ecommerce|analytics' -f partial_backup.sql production_db

-- Exclude schemas (backup everything except tenant data)
pg_dump -h localhost -U postgres -N 'tenant_*' -f shared_schema_backup.sql production_db

-- Restore single schema to different database
psql -h localhost -U postgres -d new_db -f ecommerce_schema.sql

-- Schema-only backup (no data)
pg_dump -h localhost -U postgres -n ecommerce --schema-only -f ecommerce_ddl.sql

-- Data-only for specific tenant
pg_dump -h localhost -U postgres -n tenant_acme --data-only -f acme_data.sql
```

### 11.7.3 Performance and Monitoring

```sql
-- Monitor schema sizes
SELECT 
    schemaname,
    pg_size_pretty(SUM(pg_total_relation_size(schemaname||'.'||tablename))) as total_size
FROM pg_tables 
WHERE schemaname NOT IN ('pg_catalog', 'information_schema')
GROUP BY schemaname
ORDER BY SUM(pg_total_relation_size(schemaname||'.'||tablename)) DESC;

-- Check for schema bloat (dead tuples per schema)
SELECT 
    schemaname,
    relname,
    n_dead_tup,
    n_live_tup,
    ROUND(n_dead_tup * 100.0 / NULLIF(n_live_tup + n_dead_tup, 0), 2) as dead_pct
FROM pg_stat_user_tables
WHERE schemaname LIKE 'tenant_%'
  AND n_dead_tup > 1000
ORDER BY n_dead_tup DESC;

-- Connection pooling with schemas (PgBouncer/connection pooler considerations)
-- When using schema-per-tenant, ensure connection pooler supports 
-- SET search_path or application resets schema context on checkout
```

---

## Chapter Summary

In this chapter, you learned:

1. **Schema Fundamentals**: Schemas are namespaces preventing naming collisions; always use schema-qualified names (`schema.table`) in production code; revoke `CREATE` privileges on `public` schema to prevent unauthorized object creation and schema squatting attacks.

2. **Multi-Tenant Patterns**: Schema-per-tenant provides strong isolation suitable for strict compliance requirements (customizable per tenant, easy backup/restore), but consider Row-Level Security (RLS) for high tenant counts (1000+ schemas impact catalog performance); never rely on `search_path` for tenant isolation—use explicit qualification or secure wrapper functions.

3. **Bounded Contexts**: Use schemas to enforce microservice boundaries within shared databases; create `api` schemas exposing only necessary views/functions while keeping `internal` schemas private; implement the outbox pattern in dedicated schemas for cross-service communication.

4. **Search Path Security**: Set `search_path` explicitly to prevent schema squatting attacks; use empty or minimal search paths (`"$user"`) and force fully qualified names; always specify `SET search_path` in security-definer functions to prevent privilege escalation.

5. **Privilege Management**: Grant `USAGE` on schemas to enable object access; use `ALTER DEFAULT PRIVILEGES` to automate permissions on new objects; create schema-specific roles for defense-in-depth; remember that `USAGE` on schema is required before any object-level permissions work.

6. **Cross-Schema Integration**: Foreign keys work across schemas but create tight coupling—consider triggers or application-level validation for loose coupling; use materialized views in dedicated analytics schemas to aggregate cross-schema data without impacting OLTP performance.

7. **Operational Practices**: Track per-schema migrations in a dedicated schema; use `pg_dump -n` for schema-specific backups; monitor per-schema bloat and size distribution; ensure connection poolers properly handle schema context or reset search paths on connection checkout.

---

**Next:** In Chapter 12, we will explore views and materialized views—covering when to use each type, security barrier views, refresh strategies for materialized views, and indexing strategies for derived data sets.